# XKit Lightweight Bootstrap

## Rough Notes

  - motivation
      - [#1432: Memory spike when directly visiting blogs](https://github.com/new-xkit/XKit/issues/1432)
  - cases
      - no XKit; baseline
      - XKit bootstrapped from a content script with a minimal static blacklist and eager loading of assets; effectively pre-[#1433](https://github.com/new-xkit/XKit/pull/1433)
      - XKit bootstrapped from a content script with an aggressive static blacklist and eager loading of assets; post-[#1433](https://github.com/new-xkit/XKit/pull/1433)
      - XKit bootstrapped from a content script (to a background script) with a dynamic whitelist and deferred loading of assets
          - two sub-cases will mimic configurations equivalent to the aforementioned minimal and aggressive static blacklists
          - pros
              - XKit will load in the same set of circumstances as before (since we're still bootstrapping from a content script, with the same `matches` and `exclude_matches` rules defined in the manifest)
              - we can marshal storage access via the background script (leaving stubs in `bridge.js`); this might work around [#1432](https://github.com/new-xkit/XKit/issues/1432)
              - there are probably other opportunities to share resources
          - cons
              - probably a slightly higher baseline memory usage
              - slightly more latency due to the trampoline and whitelist testing
      - XKit bootstrapped from a background script with a dynamic whitelist and deferred loading of assets
          - pros
              - we can intercept `/xkit_(editor|log|reset|update)` requests earlier, which'd permit us to do fancy things down the track (like handle an OAuth callback)
              - we can heuristically inspect and inject ourselves into blogs with custom domains (since we already have the `*://*/*` permission)
          - cons
              - we might need more permissions (like `webRequest`)
  - measurements
      - page timings
      - Chrome DevTools Performance
          - JS memory usage peak
          - total XKit-related JS execution duration
      - perhaps Chrome CPU/memory usage (via collectd)
  - targets
      - [yet-another-xkit-dev](https://yet-another-xkit-dev.tumblr.com/)  
        ![Screenshot of yet-another-xkit-dev.tumblr.com](blog.png)
          - 15 posts, 32 iframes (15 like, 5 photoset, 5 video, 5 audio, 1 controls, 1 analytics)
          - test as anonymous user
          - test with both cold and warm caches (as network requests for such a media-heavy page might mask the cost of running the extension)
      - probably not the dashboard, since we're primarily interested in exercising the iframe-heavy blog page use case
  - considerations
      - GC
      - variable VM and network performance
      - gotta nix update checks
      - whether any hypothetical performance improvements can justify a change in WebExtension permissions
      - perhaps we should also test with authenticated users and more representative XKit extension loadouts

## Frames

  - most of Tumblr (dashboard, settings, search, etc.): `https://www.tumblr.com/*`
      - frames (that don't matter):
          - `https://cookiex.ngd.yahoo.com/*` (always?)
          - `https://safe.txmblr.com/*` (for authenticated views?)
          - `https://www.tumblr.com/hsts` (sometimes?)
  - blogs: `*://*.tumblr.com/*` or `*://*/*` (for those with custom domains)
      - frames:
          - blog controls: `https://www.tumblr.com/dashboard/iframe` or `about:blank` (for unauthenticated users)
          - like control: `*://assets.tumblr.com/assets/html/like_iframe.html`
          - audio: `*://*/post/[post_id]/audio_player_iframe/[username]/*`
          - photoset: `*://*/post/[post_id]/photoset_iframe/[username]/*`
          - video: `https://www.tumblr.com/video/[username]/[post_id]/*`
      - frames (that don't matter):
          - `https://assets.tumblr.com/analytics.html`
  - other pages that contain Tumblr embeds: `*://*/*`
      - frames:
          - embed: `https://embed.tumblr.com/embed/post/*`